In [29]:
# 필요한 모듈 임포트
from functools import partial
from dotenv import load_dotenv
from langchain_together import ChatTogether

from modules import tools  # analyze_with_llm_tool 함수가 정의된 파일
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
import os

from test.memory import user_id

# 환경 변수 로드
load_dotenv()

# LLM 초기화
llm_high = ChatAnthropic(model="claude-3-5-sonnet-20240620")

llm_low = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
)
llm_low = ChatTogether(
    model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# partial을 사용하여 llm을 미리 바인딩
analyze_tool_func = partial(tools.analyze_with_llm, llm=llm_low)

# Tool 객체 생성
analysis_tool = Tool(
    name="LLM Analysis Tool",
    func=analyze_tool_func,  # partial을 사용하여 바인딩한 함수 전달
    description="요청의 유형을 분석하여 어떤 에이전트를 사용할지 결정합니다."
)


In [20]:
# LLM 설정

config = {
    "llm": {
        "provider": "together",  # 'together'를 공급자로 설정
        "config": {
            "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",  # 사용하려는 모델 지정
            "temperature": 0.2,
            "max_tokens": 1500,
        }
    },
      "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": "CSD1",
            "api_key":os.environ["QDRANT_API_KEY"],
            "url": "https://b236ccb3-b4d5-4349-a48d-6a8026f62951.us-east4-0.gcp.cloud.qdrant.io",
            "port": 6333,
            "on_disk": True
        }
    }
}

# Memory 객체 초기화
m = Memory.from_config(config)

# 사용자 정보와 메모리 추가
result = m.add("Likes to play cricket on weekends", user_id="alice", metadata={"category": "hobbies"})
print(result)  # 이 결과를 출력해봐서 데이터가 추가됐는지 확인
# 메모리 검색
related_memories = m.search(query="Help me plan my weekend.", user_id="alice")

# 검색 결과 출력
print(related_memories)


[{'id': 'cd3c2500-dcb5-48d0-91f5-da9ba0cf4c69', 'memory': 'Likes to play cricket on weekends', 'hash': '285d07801ae42054732314853e9eadd7', 'metadata': {'category': 'hobbies'}, 'score': 0.3744769, 'created_at': '2024-11-14T08:11:11.246090-08:00', 'updated_at': None, 'user_id': 'alice'}]


/tmp/ipykernel_16078/1982842456.py:1: DeprecationWarning: The current get_all API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  related_memories = m.search(query="Help me plan my weekend.", user_id="alice")


In [2]:
# LLM 설정
config = {
    "llm": {
        "provider": "together",  # 'together'를 공급자로 설정
        "config": {
            "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",  # 사용하려는 모델 지정
            "temperature": 0.2,
            "max_tokens": 1500,
        }
    },
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": "test",
            "host": "localhost",
            "port": 6333,
        }
    }
}

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

# Qdrant 클라이언트 초기화
qdrant_client = QdrantClient(
    url="https://b236ccb3-b4d5-4349-a48d-6a8026f62951.us-east4-0.gcp.cloud.qdrant.io",
    api_key=os.environ["QDRANT_API_KEY"]
)


In [ ]:

# 컬렉션 이름 설정
collection_name = "CSD1"  # 원하는 컬렉션 이름

# 컬렉션 파라미터 설정 (벡터 차원 수와 거리 계산 방식 설정)
vectors_config = VectorParams(
    size=512,  # 벡터 차원 크기 멤제로 기본이 512
    distance=Distance.COSINE  # 유사도 계산 방식 (COSINE, L2, IP 중 선택)
)

# 컬렉션 생성
qdrant_client.create_collection(
    collection_name=collection_name, 
    vectors_config=vectors_config  # 벡터 설정 추가
)

print(f"Collection '{collection_name}' created successfully.")

In [4]:
import os
from qdrant_client import QdrantClient
from qdrant_client.http.models import Filter, FieldCondition, MatchValue
import dotenv

# 환경 변수 로드
dotenv.load_dotenv()

# Qdrant 클라이언트 초기화
qdrant_client = QdrantClient(
    url="https://b236ccb3-b4d5-4349-a48d-6a8026f62951.us-east4-0.gcp.cloud.qdrant.io",
    api_key=os.environ["QDRANT_API_KEY"]
)

# 컬렉션 이름 설정
collection_name = "CSD1"

# 특정 user_id로 필터링 조건 생성
user_id = "testman1"  # 원하는 사용자 ID
filter_condition = Filter(
    must=[
        FieldCondition(
            key="user_id",  # 필터링할 필드 이름
            match=MatchValue(value=user_id)  # 매칭 조건
        )
    ]
)

# 모든 데이터를 저장할 리스트 초기화
all_points = []
next_page_offset = None

while True:
    # scroll 메서드 호출
    points, next_page_offset = qdrant_client.scroll(
        collection_name=collection_name,
        limit=100,  # 한 번에 가져올 데이터 수
        with_payload=True,  # 모든 payload 포함
        with_vectors=True,  # 벡터 포함
        offset=next_page_offset,  # 다음 페이지 오프셋
        filter=filter_condition  # 필터 조건 추가
    )

    # 가져온 포인트를 리스트에 추가
    all_points.extend(points)

    # 더 이상 가져올 데이터가 없으면 중단
    if not next_page_offset:
        break

# 결과 출력
print(f"총 {len(all_points)}개의 포인트를 가져왔습니다.")
for point in all_points:
    print(f"ID: {point.id}, Payload: {point.payload}, Vector: {point.vector}")


AssertionError: Unknown arguments: ['filter']

In [12]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import dotenv

# 환경 변수 로드
dotenv.load_dotenv()

# Qdrant 클라이언트 초기화
qdrant_client = QdrantClient(
    url="https://b236ccb3-b4d5-4349-a48d-6a8026f62951.us-east4-0.gcp.cloud.qdrant.io",
    api_key=os.environ["QDRANT_API_KEY"]
)

# 컬렉션 이름 설정
collection_name = "CSD1"

# 특정 user_id로 필터링 조건 생성
user_id = "testman1"  # 원하는 사용자 ID
scroll_filter = models.Filter(
    must=[
        models.FieldCondition(
            key="user_id",  # 필터링할 필드 이름
            match=models.MatchValue(value=user_id)  # 매칭 조건
        )
    ]
)

# 모든 데이터를 저장할 리스트 초기화
all_points = []
next_page_offset = None

while True:
    # scroll 메서드 호출
    points, next_page_offset = qdrant_client.scroll(
        collection_name=collection_name,
        limit=100,  # 한 번에 가져올 데이터 수
        with_payload=True,  # 모든 payload 포함
        with_vectors=True,  # 벡터 포함
        offset=next_page_offset,  # 다음 페이지 오프셋
        scroll_filter=scroll_filter  # 필터 조건 추가
    )

    # 가져온 포인트를 리스트에 추가
    all_points.extend(points)

    # 더 이상 가져올 데이터가 없으면 중단
    if not next_page_offset:
        break

# 결과 출력
print(f"총 {len(all_points)}개의 포인트를 가져왔습니다.")
for point in all_points:
    print(f"ID: {point.id}, Payload: {point.payload}")


총 57개의 포인트를 가져왔습니다.
ID: 0838faf0-98d9-427d-a0a1-501fa8ee71eb, Payload: {'user_id': 'testman1', 'data': 'Language of the user is English', 'hash': 'c320b2488be2d3d1f899b9893b9057dd', 'created_at': 1732206587}
ID: 1659656c-1aed-4077-8935-5e3de23f6d77, Payload: {'user_id': 'testman1', 'data': 'Grandfather talked about the beautiful night view of Seoul', 'hash': 'c8c018f6be0112b47d136523b6ea7336', 'created_at': 1732154963}
ID: 1812def5-2cc4-43b5-8c3d-d3bad536ca2c, Payload: {'user_id': 'testman1', 'data': 'Had a pleasant conversation about Seoul', 'hash': 'a029a84192c16b863e8d27e85ca1c2b0', 'created_at': 1732191555}
ID: 1a534cd8-fe25-42b3-b27f-368c7224b7e8, Payload: {'user_id': 'testman1', 'data': 'Discussed Seoul on 2024-11-21', 'hash': 'aa3972f1a1a9773bd46024a23a268f6b', 'created_at': 1732191553}
ID: 1c2a9625-d5da-4640-90ba-4699a5bb8950, Payload: {'user_id': 'testman1', 'data': '서울에 대해 이야기한 적이 있음', 'hash': 'e45704989ae81c82dad74c27537afc7c', 'created_at': 1732228725}
ID: 1dabc9f8-87ef-456

In [16]:

collections = qdrant_client.get_collections()
print(collections)


collections=[CollectionDescription(name='CSD1')]


In [2]:
from datetime import datetime
import os
from mem0 import Memory
from dotenv import load_dotenv
import time

# 환경 변수 로드
load_dotenv()

# LLM 및 Vector Store 설정
config = {
    "llm": {
        "provider": "together",  # 'together'를 공급자로 설정
        "config": {
            "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",  # 사용하려는 모델 지정
            "temperature": 0.2,
            "max_tokens": 1500,
        }
    },
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "collection_name": "CSD1",
            "api_key": os.environ["QDRANT_API_KEY"],
            "url": "https://b236ccb3-b4d5-4349-a48d-6a8026f62951.us-east4-0.gcp.cloud.qdrant.io",
            "port": 6333,
            "on_disk": True
        }
    }
}

# Memory 객체 초기화
m = Memory.from_config(config)

# 가짜 데이터 추가
fake_data = [
    {
        "role": "user",
        "content": "나는 한때 서울에 살았었지. 거기는 정말 아름다운 곳이었어"
    },
    {
        "role": "assistant",
        "content": "정말 아름다운 곳 입니다 서울, 저도 꼭 한번 보고싶네요! 서울의 어떤점이 좋으셨나요?."
    }
]

user_id1 = "testman1"
# 데이터 추가 함수
def add_fake_data(memory, user_idus, interactions):
    memory.add(interactions, user_id=user_idus)
    
#add_fake_data(m, user_id1, fake_data)

In [3]:

# 사용자 ID
user_id = "testman1"

# 타이밍 시작
start_time1 = time.time()
# 날짜/시간 데이터를 타임스탬프로 변환

start_time = datetime.fromisoformat("2024-11-19T12:00:00-08:00").timestamp()
mid_time = datetime.fromisoformat("2024-11-20T04:21:20.910935-08:00").timestamp()
end_time = datetime.fromisoformat("2024-11-22T12:00:00-08:00").timestamp()


filters = {
    "created_at": {
        "gte": start_time,  # 시작 날짜
        "lte": end_time   # 끝 날짜
    }
}

# 메모리 검색 실행
query = "내가 서울에 살았나??"
related_memories = m.search(
    query=query,   
    user_id=user_id,    # 사용자 ID    # 필터 적용
    filters=filters
)


# 타이밍 종료
end_time1 = time.time()

# 검색 결과 출력
print("Related Memories:")
print(related_memories)

# 소요된 시간 출력
elapsed_time = end_time1 - start_time1
print(f"Time taken: {elapsed_time:.4f} seconds")


Related Memories:
[{'id': '53918bc1-626e-4046-9446-451d812f6441', 'memory': 'The user has lived in Seoul before', 'hash': '9b6fc0a0a4f66991caee7889a2654d09', 'metadata': None, 'score': 0.5656013, 'created_at': 1732115708, 'updated_at': None, 'user_id': 'testman1'}, {'id': 'd16c4e86-4f79-4f9a-afd6-0b2fd69c108a', 'memory': 'The user thinks Seoul is a beautiful place', 'hash': '8de7fd4a90cb02c1d922552345b184c1', 'metadata': None, 'score': 0.48030862, 'created_at': 1732115708, 'updated_at': None, 'user_id': 'testman1'}]
Time taken: 1.7823 seconds


/tmp/ipykernel_132832/1785788346.py:22: DeprecationWarning: The current get_all API output format is deprecated. To use the latest format, set `api_version='v1.1'`. The current format will be removed in mem0ai 1.1.0 and later versions.
  related_memories = m.search(
